In [46]:
# import and install required libraries
!pip install deep_translator

import pandas as pd
import numpy as np
import time
import os
import datetime
from deep_translator import GoogleTranslator

Defaulting to user installation because normal site-packages is not writeable


In [2]:
googledf = pd.read_csv(r'ScrapingExtract\COO_CSI_venue.csv')

### Merging sub-categories

In [4]:
merged_rows = googledf.groupby(['NAME', 'CATEGORY', 'LATITUDE', 'LONGITUDE','POSTAL_CODE', 'ADDRESS', 'CITY', 
                                'PROVINCE', 'COUNTRY','ACTIVE', 'WEBSITE', 'PHONE']
                               ,as_index=False,dropna=False).agg({'TAGS': 'first','SUB_CATEGORY': ', '.join})

In [5]:
columns = ['NAME', 'CATEGORY', 'SUB_CATEGORY', 'TAGS', 'LATITUDE', 'LONGITUDE',
       'POSTAL_CODE', 'ADDRESS', 'CITY', 'PROVINCE', 'COUNTRY',
       'ACTIVE', 'WEBSITE', 'PHONE']
merged_rows = merged_rows[columns]

In [6]:
googledf = merged_rows

In [7]:
columns = ['NAME', 'CATEGORY', 'SUB_CATEGORY', 'TAGS', 'LATITUDE', 'LONGITUDE', 'POSTAL_CODE', 'ADDRESS', 'CITY', 'PROVINCE',
           'COUNTRY', 'ACTIVE', 'WEBSITE', 'PHONE']
googledf = googledf[columns]

### Data Imputation

In [12]:
googledf['ACTIVE'].fillna("OPERATIONAL",inplace=True)

In [14]:
googledf["PROVINCE"] = googledf["PROVINCE"].apply(lambda x: x.replace("Ontario", "ON"))

In [15]:
googledf["PROVINCE"] = googledf["PROVINCE"].apply(lambda x: x.replace("Québec", "QC"))

In [16]:
googledf["PROVINCE"] = googledf["PROVINCE"].apply(lambda x: x.replace("Quebec", "QC"))

In [17]:
googledf["PROVINCE"] = googledf["PROVINCE"].apply(lambda x: x.replace("New York", "NY"))

In [18]:
googledf["ACTIVE"] = googledf["ACTIVE"].apply(lambda x: x.replace("OPERATIONAL", "Yes"))

In [19]:
googledf["ACTIVE"] = googledf["ACTIVE"].apply(lambda x: x.replace("CLOSED_TEMPORARILY", "No"))

In [20]:
googledf["ACTIVE"] = googledf["ACTIVE"].apply(lambda x: x.replace("CLOSED_PERMANENTLY", "No"))

### Removing city from Address

In [22]:
googledf[googledf["ADDRESS"].str.count(",")>=2]

,NAME,CATEGORY,SUB_CATEGORY,TAGS,LATITUDE,LONGITUDE,POSTAL_CODE,ADDRESS,CITY,PROVINCE,COUNTRY,ACTIVE,WEBSITE,PHONE


In [23]:
# Removing city from address

for index, row in googledf.iterrows():
    address = row['ADDRESS']
    updated_address = address.rsplit(",", 1)[0].strip()
    googledf.at[index, 'ADDRESS'] = updated_address

### Address Translation

In [26]:
fr_googledf = googledf[['ADDRESS']].copy()

In [28]:
translations = {}
for column in fr_googledf.columns:
    # unique elements of the column
    unique_elements = fr_googledf[column].unique()
    for element in unique_elements:
        # add translation to the dictionary
        translations[element] = GoogleTranslator(source='auto', target='fr').translate(element)
        time.sleep(1)

In [29]:
fr_googledf.replace(translations, inplace = True)

In [31]:
fr_googledf.rename(columns=lambda x: GoogleTranslator(source='auto', target='fr').translate(x), inplace=True)

In [33]:
final = pd.concat([googledf, fr_googledf], axis=1)

,NAME,CATEGORY,SUB_CATEGORY,TAGS,LATITUDE,LONGITUDE,POSTAL_CODE,ADDRESS,CITY,PROVINCE,COUNTRY,ACTIVE,WEBSITE,PHONE,ADRESSE
0,Canadian Sport Martial Arts Academy - CSMA Bar...,venue,"arts, martial arts","point_of_interest, health, establishment",45.260385,-75.768416,K2J 0V2,4100 Strandherd Dr Suite 213,Ottawa,ON,Canada,Yes,http://www.csma.ca/,(613) 831-8085,4100 promenade Strandherd Suite 213
1,Clarke Fields Park,venue,parks,"park, tourist_attraction, point_of_interest, e...",45.261994,-75.768682,K2J,93 Houlahan St,Ottawa,ON,Canada,Yes,https://ottawa.ca/en/residents/recreation-and-...,(613) 580-2400,93 rue Houlahan
2,Dr. Soyoun Kim's Piano Lessons,venue,music,"electronics_store, school, home_goods_store, s...",45.251601,-75.758516,K2J 3V5,133 Alamo St,Ottawa,ON,Canada,Yes,NaN,(613) 301-1338,"133, rue Alamo"
3,La Bomba,venue,food,"meal_takeaway, restaurant, point_of_interest, ...",45.260318,-75.768296,K2J 0V2,4100 Strandherd Dr unit 109,Ottawa,ON,Canada,Yes,NaN,NaN,4100 promenade Strandherd unité 109
4,Montessori By BrightPath - Strandherd,venue,school,"school, point_of_interest, establishment",45.260585,-75.767795,K2J 0V2,4100 Strandherd Dr,Ottawa,ON,Canada,Yes,NaN,(613) 843-4100,"4100, promenade Strandherd"
5,Ottawa Christian School,venue,school,"primary_school, school, point_of_interest, est...",45.261390,-75.765706,K2J 0V2,255 Tartan Dr,Ottawa,ON,Canada,Yes,http://www.ocschool.org/,(613) 825-3000,"255, promenade Tartan"
6,Pinelopi's Greek Kitchen,venue,"food, fine dining, pub","restaurant, point_of_interest, food, establish...",45.260318,-75.768296,K2J 0V2,4100 Strandherd Dr #109,Ottawa,ON,Canada,Yes,https://www.pinelopis.com/,(613) 843-1111,4100 promenade Strandherd #109
7,Sequoia Community Church,venue,"place of worship, community, church","church, place_of_worship, point_of_interest, e...",45.261403,-75.765677,K2J 0V2,255 Tartan Dr,Ottawa,ON,Canada,Yes,https://www.sequoiachurch.org/,(613) 224-0700,"255, promenade Tartan"
8,South Nepean Satellite Community Health Centre,venue,community,"point_of_interest, establishment",45.260521,-75.767829,K2J 0V2,4100 Strandherd Dr,Ottawa,ON,Canada,Yes,NaN,(613) 288-2820,"4100, promenade Strandherd"
9,St. Juan Diego School,venue,school,"school, point_of_interest, establishment",45.249259,-75.760333,K2J 3V5,135 Main Halyard Ln,Ottawa,ON,Canada,Yes,https://jua.ocsb.ca/,NaN,135 Drisse principale Ln


In [34]:
final.rename(columns = {'ADRESSE':'ADDRESS_FR'}, inplace = True)

In [35]:
columns = ['NAME', 'CATEGORY', 'SUB_CATEGORY', 'TAGS', 'LATITUDE', 'LONGITUDE',
       'POSTAL_CODE', 'ADDRESS', 'ADDRESS_FR', 'CITY', 'PROVINCE', 'COUNTRY',
       'ACTIVE', 'WEBSITE', 'PHONE']
final = final[columns]

In [44]:
# Create the folder "TransformedFinal" if it doesn't exist
if not os.path.exists("TransformedFinal"):
    os.makedirs("TransformedFinal")

csv_file = os.path.join("TransformedFinal", "COO_CSI_VENUE_TRANSFORMED"+datetime.datetime.now().strftime("%B_%d_%Y-%I%M%S")+".csv")
final.to_csv(csv_file,index=False)